## 다음뉴스 크롤링

In [1]:
from urllib.request import urlopen
import bs4
import requests
import pandas as pd

# (1)

In [2]:
url = 'https://news.daum.net'
html = urlopen(url)
bs_obj = bs4.BeautifulSoup(html, 'html.parser')

In [3]:
# 섹션 리스트
lis = bs_obj.find('ul', {'class':'gnb_comm'}).findAll('li')
sections = []
links = []
for i in range(1, 7):
    sections.append(lis[i].text)
    links.append(url+lis[i].find('a')['href'])

In [4]:
section_menu_df = pd.DataFrame({'section':sections, 'link':links})
section_menu_df.to_csv('crawl_data/daum_section_menu.csv', index=0)

# (2)

In [5]:
section_menu_df = pd.read_csv('crawl_data/daum_section_menu.csv')

In [6]:
def get_topic(url, section):
    headers = {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36'}
    res = requests.get(url, headers=headers)
    html = res.text
    bs_obj = bs4.BeautifulSoup(html, 'html.parser')
    
    # 토픽 리스트
    topic_list = bs_obj.select('ul.list_mainnews li')
    topic_title = []
    topic_link = []

    for topic in topic_list:
        a = topic.select_one('a.link_txt')
        topic_title.append(a.text)
        topic_link.append(a['href'])
            
    return {'topic':topic_title, 'url':topic_link, 'section':section}

In [7]:
all_topic_news_df = pd.DataFrame({
    'topic' : [],
    'url' : [],
    'section' : []
})

for i in range(6):
    df = pd.DataFrame(get_topic(section_menu_df['link'][i], section_menu_df['section'][i]))
    all_topic_news_df = pd.concat([all_topic_news_df, df], axis=0, ignore_index=True)
    
all_topic_news_df.to_csv('crawl_data/daum_news_all_topic.csv', index=0)

# (3)

In [8]:
all_topic_news_df = pd.read_csv('crawl_data/daum_news_all_topic.csv')

In [9]:
def get_sub_news_info(url):
    headers = {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36'}
    res = requests.get(url, headers=headers)
    html = res.text
    bs_obj = bs4.BeautifulSoup(html, 'html.parser')
    
    paper_list, title_list, datetime_list, writing_list, writer_list = [[] for _ in range(5)]
    
    # 언론사
    paper_list.append(bs_obj.select_one('#kakaoServiceLogo').text)
    # 제목
    title_list.append(bs_obj.select_one('div.head_view h3').text)
    # 작성시간
    datetime_list.append(bs_obj.select_one('span.num_date').text)
    # 내용
    writing_list.append(bs_obj.select_one('div.article_view section').text.replace('\n', ' ').strip())
    # 작성자
    writer_list.append(bs_obj.select_one('div.info_view span:first-child').text)
    
    sub_news_dict = {
        'paper' : paper_list,
        'title' : title_list,
        'datetime' : datetime_list,
        'writing' : writing_list,
        'writer' : writer_list
    }
    return sub_news_dict

In [10]:
all_sub_news_info_df = pd.DataFrame({
    'paper' : [],
    'title' : [],
    'datetime' : [],
    'writing' : [],
    'writer' : []
})

for url in all_topic_news_df['url']:
    df = pd.DataFrame(get_sub_news_info(url))
    all_sub_news_info_df = pd.concat([all_sub_news_info_df, df], axis=0, ignore_index=True)
    
all_sub_news_info_df.to_csv('crawl_data/daum_sub_news_section.csv', index=0)